In [1]:
import os
import numpy as np
import torch
import torch.optim as optim
import torch.distributions as D
from src.Tmy_svgd import tmySVGD
from src.svgd import SVGD
from src.gsvgd import FullGSVGDBatch
from src.kernel import RBF, BatchRBF
from src.utils import plot_particles

from src.manifold import Grassmann
from src.s_svgd import SlicedSVGD
from src.mysvgd import mySVGD

from scipy.spatial.distance import pdist, squareform
from scipy.stats import multivariate_normal, gaussian_kde, norm

import pickle
import argparse
import time
import torch
import torch.optim as optim
import torch.autograd as autograd
import autograd.numpy as np
from tqdm import tqdm, trange

import torch.distributions as D
import matplotlib.pyplot as plt

/home/zhoujk/anaconda3/envs/AG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description='Running xshaped experiment.')
d = 10
parser.add_argument('--dim', type=int, default=d, help='dimension')
s = 20000
parser.add_argument('--effdim', type=int, default=-1, help='dimension')
parser.add_argument('--lr', type=float, default=0.01, help='learning rate')
parser.add_argument('--delta', type=float,default=0.01,help='stepsize for projections')
parser.add_argument('--T', type=float, default=1e-4, help='noise multiplier for projections')
parser.add_argument('--lr_g', type=float, default=0.1, help='learning rate for g')
parser.add_argument('--nparticles', type=int,default=s, help='no. of particles')
parser.add_argument('--epochs', type=int, default=10000, help='no. of epochs')
parser.add_argument('--metric', type=str, default="energy", help='distance metric')
parser.add_argument('--noise', type=str, default="True", help='whether to add noise')
parser.add_argument('--kernel', type=str, default="rbf", help='kernel')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--seed', type=int, default=235, help='random seed')
parser.add_argument('--suffix', type=str, default="", help='suffix for res folder')
parser.add_argument('--m', type=int, help='no. of projections')
parser.add_argument('--save_every', type=int, default=200, help='step intervals to save particles')
parser.add_argument('--method', type=str, default="all", help='which method to use')

_StoreAction(option_strings=['--method'], dest='method', nargs=None, const=None, default='all', type=<class 'str'>, choices=None, required=False, help='which method to use', metavar=None)

In [3]:
args = parser.parse_args([])
dim = args.dim
lr = args.lr
lr_gsvgd = args.lr
delta = args.delta
T = args.T
nparticles = args.nparticles
epochs = args.epochs
seed = args.seed
eff_dims = [args.effdim] if args.effdim > 0 else [1, 2, 5]
save_every = args.save_every # save metric values
print(f"Running for dim: {dim}, lr: {lr}, nparticles: {nparticles}")

Running for dim: 10, lr: 0.01, nparticles: 20000


In [4]:
device = torch.device(f'cuda:{args.gpu}' if args.gpu != -1 else 'cpu')

metric = args.metric

results_folder = f"./res/gaussian{args.suffix}/{args.kernel}_epoch{epochs}_lr{lr}_delta{delta}_n{nparticles}_dim{dim}"
results_folder = f"{results_folder}/seed{seed}"

In [5]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

if args.kernel == "rbf":
    Kernel = RBF
    BatchKernel = BatchRBF

In [6]:
print(f"Device: {device}")
torch.manual_seed(seed)

## target density
means = torch.zeros(dim, device=device)
'''
torch.manual_seed(0)

A = torch.randn(dim,dim).to('cuda') * 0.9
A = torch.matmul(A, A.T)

m = torch.max(A) 
B = torch.eye(dim).to('cuda') * m + 0.1
diag = torch.diag(A)
cov = A + B '''

cov = torch.eye(dim, device=device)

distribution = D.MultivariateNormal(means.to(device), cov)

# sample from target (for computing metric)
x_target = distribution.sample((nparticles, ))
# sample from variational density
torch.manual_seed(235)
x_init = 2 + np.sqrt(2) * torch.randn(nparticles, *distribution.event_shape, device=device)

Device: cuda:0


In [7]:
## SVGD
if args.method in ["SVGD", "all"]:
    print("Running SVGD")
    # sample from variational density
    x = x_init.clone().to(device)
    kernel = Kernel(method="med_heuristic")
    svgd = SVGD(distribution, kernel, optim.Adam([x], lr=lr), device=device)
    start = time.time()
    svgd.fit(x, epochs, verbose=True, save_every=save_every)
    elapsed_time_svgd = time.time() - start



Running SVGD


100%|██████████| 10000/10000 [20:28<00:00,  8.14it/s]


In [8]:
sq_dist = pdist(x.cpu().numpy())

pairwise_dists = squareform(sq_dist)**2
h  = -1
if h < 0: # if h < 0, using median trick
    h = np.median(pairwise_dists)  
    h = np.sqrt(0.5 * h / np.log(x.shape[0]+1))  

h

0.9114885914975139

In [9]:
mean_norm= torch.zeros(s, 1)
for i in range(x.shape[0]):
    mean_norm[i] = torch.norm(x[i,:])

mean = torch.mean(x, 0)
mean_norm

tensor([[3.2144],
        [3.1782],
        [3.2260],
        ...,
        [3.2240],
        [0.6471],
        [3.2345]])

In [10]:
print(torch.mean(mean_norm))
print(torch.max(mean_norm))

tensor(2.8721)
tensor(3.3110)


In [11]:
torch.mean(mean_norm)  * (np.exp(1.5)+1)

tensor(15.7442)

In [12]:
cov_svgd = torch.cov(x.T)
cov_svgd

tensor([[ 8.4792e-01,  2.9419e-06,  1.3890e-04,  2.0322e-04,  1.5244e-04,
          1.3856e-06,  1.7259e-05,  6.7750e-05, -1.4528e-05, -4.2218e-05],
        [ 2.9419e-06,  8.4838e-01,  1.4927e-04, -3.6418e-05,  2.2723e-05,
          8.4771e-05,  1.4161e-04,  1.8221e-04, -1.5658e-05,  1.2523e-04],
        [ 1.3890e-04,  1.4927e-04,  8.4812e-01,  1.2845e-04, -2.0085e-05,
         -2.3613e-04,  1.0015e-04, -9.3802e-06,  2.9389e-05,  1.1236e-04],
        [ 2.0322e-04, -3.6418e-05,  1.2845e-04,  8.4835e-01, -5.5547e-05,
          1.7215e-04,  8.4654e-05,  2.2196e-05,  8.4594e-05,  4.0353e-05],
        [ 1.5244e-04,  2.2723e-05, -2.0085e-05, -5.5547e-05,  8.4809e-01,
          1.6993e-04,  9.9048e-05, -1.7560e-04,  4.5739e-05,  1.4798e-04],
        [ 1.3856e-06,  8.4771e-05, -2.3613e-04,  1.7215e-04,  1.6993e-04,
          8.4808e-01,  4.2124e-05,  8.8210e-05,  2.3363e-05, -4.4628e-05],
        [ 1.7259e-05,  1.4161e-04,  1.0015e-04,  8.4654e-05,  9.9048e-05,
          4.2124e-05,  8.4833e-0

In [13]:
h = torch.tensor(h).to(device)

def M_function(xi):
    x = torch.arange(0.00001, 1000, 0.001).to(device)
    f1 = torch.pow(xi - x, 2)
    f2 = torch.pow(xi, 2)
    f =torch.div(torch.exp(-f1 / h) - torch.exp(-f2 / h), 2*torch.abs(x))

    
    return f
list = []
for i in range(x.shape[0]):
    s = M_function(x[i,:])
    list.append(s[0].item())

RuntimeError: The size of tensor a (10) must match the size of tensor b (1000000) at non-singleton dimension 0

In [ ]:
np.max(list)